## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,91.40,57,0,20.71,clear sky,2021-01-28 20:07:50
1,1,Krasnoselkup,RU,65.7000,82.4667,-32.85,80,100,14.83,overcast clouds,2021-01-28 20:13:52
2,2,Punta Arenas,CL,-53.1500,-70.9167,48.20,57,75,23.02,broken clouds,2021-01-28 20:07:58
3,3,Novyy Urengoy,RU,66.0833,76.6333,-32.08,83,100,11.21,overcast clouds,2021-01-28 20:10:53
4,4,Nevers,FR,46.9167,3.3333,53.60,94,75,12.66,broken clouds,2021-01-28 20:13:53


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,91.40,57,0,20.71,clear sky,2021-01-28 20:07:50
6,6,Kavieng,PG,-2.5744,150.7967,82.83,76,59,9.37,broken clouds,2021-01-28 20:13:00
8,8,Kapaa,US,22.0752,-159.3190,75.20,78,75,17.27,broken clouds,2021-01-28 20:12:02
13,13,Hilo,US,19.7297,-155.0900,75.20,69,75,5.75,light rain,2021-01-28 20:12:14
14,14,Rikitea,PF,-23.1203,-134.9692,79.68,68,89,6.31,overcast clouds,2021-01-28 20:13:54
18,18,Ribeira Grande,PT,38.5167,-28.7000,64.40,88,75,28.77,broken clouds,2021-01-28 20:09:18
21,21,Cabatuan,PH,10.8794,122.4860,72.64,88,47,13.73,scattered clouds,2021-01-28 20:13:55
23,23,Calama,CL,-22.4667,-68.9333,75.20,13,0,23.02,clear sky,2021-01-28 20:13:56
24,24,Manggar,ID,-2.8833,108.2667,77.07,88,99,8.72,overcast clouds,2021-01-28 20:13:07
25,25,Umm Lajj,SA,25.0213,37.2685,70.59,59,44,7.43,scattered clouds,2021-01-28 20:13:56


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,91.40,clear sky,-22.9661,-42.0278,
6,Kavieng,PG,82.83,broken clouds,-2.5744,150.7967,
8,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,
13,Hilo,US,75.20,light rain,19.7297,-155.0900,
14,Rikitea,PF,79.68,overcast clouds,-23.1203,-134.9692,
18,Ribeira Grande,PT,64.40,broken clouds,38.5167,-28.7000,
21,Cabatuan,PH,72.64,scattered clouds,10.8794,122.4860,
23,Calama,CL,75.20,clear sky,-22.4667,-68.9333,
24,Manggar,ID,77.07,overcast clouds,-2.8833,108.2667,
25,Umm Lajj,SA,70.59,scattered clouds,25.0213,37.2685,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,91.40,clear sky,-22.9661,-42.0278,Pousada Porto Praia
6,Kavieng,PG,82.83,broken clouds,-2.5744,150.7967,Nusa Island Retreat
8,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
13,Hilo,US,75.20,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Rikitea,PF,79.68,overcast clouds,-23.1203,-134.9692,Pension Maro'i
18,Ribeira Grande,PT,64.40,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
21,Cabatuan,PH,72.64,scattered clouds,10.8794,122.4860,ROREM'S BOARDING HOUSE
23,Calama,CL,75.20,clear sky,-22.4667,-68.9333,Park Hotel Calama
24,Manggar,ID,77.07,overcast clouds,-2.8833,108.2667,Guest Hotel
25,Umm Lajj,SA,70.59,scattered clouds,25.0213,37.2685,Moon light Furnished Units


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))